In [2]:
import json
from functools import lru_cache
from typing import List

import pandas as pd
import requests
from pydantic import BaseModel, confloat, AnyUrl
from ratelimit import RateLimitException, limits
from tenacity import *
from yarl import URL

In [3]:
API_ENDPOINT = URL("https://api.coingecko.com/api/v3")

In [4]:
@retry(
    retry=retry_if_exception_type(requests.exceptions.RequestException),
    wait=wait_exponential(max=60),
    stop=stop_after_attempt(3),
)
@retry(
    retry=retry_if_exception_type(RateLimitException),
    wait=wait_fixed(60),
    stop=stop_after_delay(360),
)
@limits(calls=50, period=60)
def call_get(url: URL, params: dict) -> str:
    return requests.get(url.update_query(params)).text

In [13]:
def get_coin_list() -> List[str]:
    return [
        result for result in json.loads(
            call_get(API_ENDPOINT / "coins/list", dict())
        )
    ]

In [14]:
# dict_market_data = {'vs_currency' : 'usd', 'per_page' : '250'}

In [15]:
pd.json_normalize(get_coin_list())

,id,symbol,name
0,,aapt,AAptitude
1,01coin,zoc,01coin
2,0-5x-long-algorand-token,algohalf,0.5X Long Algorand Token
3,0-5x-long-altcoin-index-token,althalf,0.5X Long Altcoin Index Token
4,0-5x-long-ascendex-token-token,asdhalf,0.5X Long AscendEx Token Token
...,...,...,...
13086,zyro,zyro,Zyro
13087,zyrri,zyr,Zyrri
13088,zytara-dollar,zusd,Zytara Dollar
13089,zyx,zyx,ZYX


In [45]:
class MarketDataInfo(BaseModel):
    id: str
    symbol: str
    current_price: float
    market_cap: int
    image: AnyUrl



In [47]:
def parse_result(result) -> MarketDataInfo:
    return MarketDataInfo(
        id=result["id"],
        symbol=result["symbol"],
        current_price=result["current_price"],
        market_cap=result["market_cap"],
        image=result["image"]
    )

In [48]:
def get_market_data(page: int) -> List[str]:
    return [
        parse_result(result)
        for result in json.loads(
            call_get(API_ENDPOINT / "coins/markets", dict_market_data)
        )
    ]

In [50]:
def create_df() -> pd.DataFrame:
    return pd.DataFrame([item.dict() for item in get_market_data()])